# IVFPQ Algorithm Examples in FAISS

This notebook demonstrates the IVFPQ (Inverted File with Product Quantization) algorithm in FAISS with various parameter combinations.

## Key Parameters

### Index Creation Parameters
- **nlist**: Number of clusters (Voronoi cells) for the inverted file
- **m**: Number of subquantizers for Product Quantization (must divide dimension d)
- **nbits**: Number of bits per subquantizer code (typically 8, giving 256 centroids per subspace)

### Search-time Parameters  
- **nprobe**: Number of clusters to search

## How IVFPQ Works

1. **Training Phase**:
   - K-means clustering creates `nlist` cluster centroids (coarse quantizer)
   - Product Quantizer learns `m` codebooks with `2^nbits` centroids each

2. **Adding Vectors**:
   - Each vector is assigned to its nearest cluster
   - The residual (vector - centroid) is encoded using PQ into `m` bytes (with nbits=8)

3. **Searching**:
   - Query is compared to cluster centroids, selecting `nprobe` nearest clusters
   - Distance tables are precomputed for fast PQ distance estimation
   - Approximate distances computed using lookup tables

## Memory Advantage

IVFPQ uses much less memory than IVFFlat:
- IVFFlat stores full vectors: `n × d × 4` bytes  
- IVFPQ stores PQ codes: `n × m` bytes (with nbits=8)

In [ ]:
import numpy as np
import faiss
import time
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Dataset Generation

We'll create a synthetic dataset for our experiments. Note: dimension `d` must be divisible by `m` (number of subquantizers).

In [ ]:
def generate_dataset(nb, nq, d):
    """
    Generate random database and query vectors.
    
    Args:
        nb: Number of database vectors
        nq: Number of query vectors  
        d: Vector dimension
    
    Returns:
        xb: Database vectors (nb x d)
        xq: Query vectors (nq x d)
    """
    xb = np.random.random((nb, d)).astype('float32')
    xq = np.random.random((nq, d)).astype('float32')
    return xb, xq

# Dataset parameters
# d=128 is divisible by many values: 1, 2, 4, 8, 16, 32, 64, 128
nb = 100000   # Database size
nq = 1000     # Number of queries
d = 128       # Vector dimension (must be divisible by m)
k = 10        # Number of nearest neighbors to find

print(f"Generating dataset: {nb:,} database vectors, {nq:,} queries, dimension {d}")
print(f"Valid m values (must divide {d}): {[i for i in [1,2,4,8,16,32,64] if d % i == 0]}")
xb, xq = generate_dataset(nb, nq, d)
print(f"Database shape: {xb.shape}, Query shape: {xq.shape}")

## 2. Ground Truth Computation

Compute exact nearest neighbors using brute-force search for recall evaluation.

In [ ]:
def compute_ground_truth(xb, xq, k):
    """Compute exact nearest neighbors using IndexFlatL2."""
    index_flat = faiss.IndexFlatL2(xb.shape[1])
    index_flat.add(xb)
    distances_gt, labels_gt = index_flat.search(xq, k)
    return distances_gt, labels_gt

def compute_recall(labels_gt, labels_approx, k):
    """Compute recall@k: fraction of true neighbors found."""
    n = labels_gt.shape[0]
    recall = 0.0
    for i in range(n):
        gt_set = set(labels_gt[i, :k])
        approx_set = set(labels_approx[i, :k])
        recall += len(gt_set & approx_set) / k
    return recall / n

print("Computing ground truth with brute-force search...")
start = time.time()
distances_gt, labels_gt = compute_ground_truth(xb, xq, k)
gt_time = time.time() - start
print(f"Ground truth computed in {gt_time:.2f} seconds")

## 3. Helper Functions for IVFPQ Experiments

In [ ]:
def build_ivfpq_index(xb, nlist, m, nbits=8):
    """
    Build IVFPQ index with given parameters.
    
    Args:
        xb: Database vectors
        nlist: Number of IVF clusters
        m: Number of PQ subquantizers
        nbits: Bits per subquantizer (default 8)
    
    Returns:
        index: Built IVFPQ index
        train_time: Time taken to train (seconds)
        add_time: Time taken to add vectors (seconds)
    """
    d = xb.shape[1]
    
    # Create quantizer (flat index for centroids)
    quantizer = faiss.IndexFlatL2(d)
    
    # Create IVFPQ index
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)
    
    # Train the index
    start = time.time()
    index.train(xb)
    train_time = time.time() - start
    
    # Add vectors
    start = time.time()
    index.add(xb)
    add_time = time.time() - start
    
    return index, train_time, add_time

def search_ivfpq_index(index, xq, k, nprobe):
    """
    Search IVFPQ index with given nprobe.
    
    Returns:
        distances: Distance array
        labels: Label array
        search_time: Time taken to search (seconds)
    """
    index.nprobe = nprobe
    
    start = time.time()
    distances, labels = index.search(xq, k)
    search_time = time.time() - start
    
    return distances, labels, search_time

def estimate_memory_usage_ivfpq(n, d, nlist, m, nbits=8):
    """Estimate memory usage of IVFPQ index in MB."""
    # Centroid storage: nlist * d * 4 bytes
    centroid_mem = nlist * d * 4
    
    # PQ codebook: m * 2^nbits * (d/m) * 4 bytes
    codebook_mem = m * (2**nbits) * (d // m) * 4
    
    # PQ codes: n * m * ceil(nbits/8) bytes
    bytes_per_code = (nbits + 7) // 8
    code_mem = n * m * bytes_per_code
    
    # ID storage: n * 8 bytes
    id_mem = n * 8
    
    total_mb = (centroid_mem + codebook_mem + code_mem + id_mem) / (1024 * 1024)
    return total_mb

def estimate_memory_usage_flat(n, d):
    """Estimate memory usage of IVFFlat for comparison."""
    return (n * d * 4 + n * 8) / (1024 * 1024)

## 4. Experiment 1: Effect of m (Number of Subquantizers)

The parameter `m` controls the compression ratio:
- Higher m → Better accuracy (more bits to represent each vector)
- Lower m → More compression, less memory, but lower recall
- `m` must divide the dimension `d`
- With nbits=8, each vector is encoded in `m` bytes

In [ ]:
# Test different values of m (subquantizers)
m_values = [4, 8, 16, 32, 64]  # Must divide d=128
nlist_fixed = 256
nprobe_fixed = 16
nbits_fixed = 8

results_m = []

print("Experiment 1: Varying m (number of subquantizers)")
print(f"Fixed: nlist={nlist_fixed}, nprobe={nprobe_fixed}, nbits={nbits_fixed}\n")
print(f"{'m':>4} {'d/m':>6} {'Bytes/vec':>10} {'Train(s)':>10} {'Search(ms)':>12} {'Recall':>10} {'Mem(MB)':>10}")
print("-" * 70)

for m in m_values:
    # Build index
    index, train_time, add_time = build_ivfpq_index(xb, nlist_fixed, m, nbits_fixed)
    
    # Search
    distances, labels, search_time = search_ivfpq_index(index, xq, k, nprobe_fixed)
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    memory = estimate_memory_usage_ivfpq(nb, d, nlist_fixed, m, nbits_fixed)
    bytes_per_vec = m  # With nbits=8
    subvec_dim = d // m
    
    results_m.append({
        'm': m,
        'subvec_dim': subvec_dim,
        'bytes_per_vec': bytes_per_vec,
        'train_time': train_time,
        'add_time': add_time,
        'search_time_ms': search_time * 1000,
        'recall': recall,
        'memory_mb': memory
    })
    
    print(f"{m:>4} {subvec_dim:>6} {bytes_per_vec:>10} {train_time:>10.2f} {search_time*1000:>12.2f} {recall:>10.4f} {memory:>10.1f}")

df_m = pd.DataFrame(results_m)

# Compare with IVFFlat memory
flat_memory = estimate_memory_usage_flat(nb, d)
print(f"\nFor comparison, IVFFlat would use: {flat_memory:.1f} MB")

In [ ]:
# Visualize effect of m
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recall vs m
ax1 = axes[0, 0]
ax1.plot(df_m['m'], df_m['recall'], 'o-', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('m (subquantizers)')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs m')
ax1.set_xscale('log', base=2)
ax1.set_ylim([0, 1.02])
ax1.grid(True, alpha=0.3)

# Memory vs m (including IVFFlat baseline)
ax2 = axes[0, 1]
ax2.plot(df_m['m'], df_m['memory_mb'], 'o-', linewidth=2, markersize=8, color='#9b59b6', label='IVFPQ')
ax2.axhline(y=flat_memory, color='#e74c3c', linestyle='--', linewidth=2, label='IVFFlat')
ax2.set_xlabel('m (subquantizers)')
ax2.set_ylabel('Memory (MB)')
ax2.set_title('Memory Usage vs m')
ax2.set_xscale('log', base=2)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Search time vs m
ax3 = axes[1, 0]
ax3.plot(df_m['m'], df_m['search_time_ms'], 'o-', linewidth=2, markersize=8, color='#e74c3c')
ax3.set_xlabel('m (subquantizers)')
ax3.set_ylabel('Search Time (ms)')
ax3.set_title('Search Time vs m')
ax3.set_xscale('log', base=2)
ax3.grid(True, alpha=0.3)

# Compression ratio vs recall
ax4 = axes[1, 1]
compression_ratio = (d * 4) / df_m['bytes_per_vec']
ax4.plot(compression_ratio, df_m['recall'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax4.set_xlabel('Compression Ratio (original / compressed)')
ax4.set_ylabel('Recall@10')
ax4.set_title('Recall vs Compression Ratio')
ax4.grid(True, alpha=0.3)

# Add annotations
for i, (ratio, recall, m) in enumerate(zip(compression_ratio, df_m['recall'], df_m['m'])):
    ax4.annotate(f'm={m}', (ratio, recall), textcoords="offset points", xytext=(5, 5), fontsize=9)

plt.suptitle(f'Effect of m (nlist={nlist_fixed}, nprobe={nprobe_fixed})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Experiment 2: Effect of nlist (Number of Clusters)

Same as IVFFlat, nlist controls the coarse partitioning:
- Higher nlist → Faster search (fewer vectors per cluster)
- Lower nlist → Better recall (less boundary effects)
- Requires more training data with higher nlist

In [ ]:
# Test different values of nlist
nlist_values = [32, 64, 128, 256, 512, 1024]
m_fixed = 16
nprobe_fixed = 16

results_nlist = []

print("Experiment 2: Varying nlist")
print(f"Fixed: m={m_fixed}, nprobe={nprobe_fixed}\n")
print(f"{'nlist':>8} {'Train(s)':>10} {'Add(s)':>8} {'Search(ms)':>12} {'Recall':>10} {'Vecs/List':>12}")
print("-" * 66)

for nlist in nlist_values:
    # Build index
    index, train_time, add_time = build_ivfpq_index(xb, nlist, m_fixed)
    
    # Search
    distances, labels, search_time = search_ivfpq_index(index, xq, k, nprobe_fixed)
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    vecs_per_list = nb / nlist
    
    results_nlist.append({
        'nlist': nlist,
        'train_time': train_time,
        'add_time': add_time,
        'search_time_ms': search_time * 1000,
        'recall': recall,
        'vecs_per_list': vecs_per_list
    })
    
    print(f"{nlist:>8} {train_time:>10.2f} {add_time:>8.2f} {search_time*1000:>12.2f} {recall:>10.4f} {vecs_per_list:>12.0f}")

df_nlist = pd.DataFrame(results_nlist)

In [ ]:
# Visualize effect of nlist
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recall vs nlist
ax1 = axes[0, 0]
ax1.plot(df_nlist['nlist'], df_nlist['recall'], 'o-', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('nlist (clusters)')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs nlist')
ax1.set_xscale('log', base=2)
ax1.set_ylim([0, 1.02])
ax1.grid(True, alpha=0.3)

# Search time vs nlist
ax2 = axes[0, 1]
ax2.plot(df_nlist['nlist'], df_nlist['search_time_ms'], 'o-', linewidth=2, markersize=8, color='#e74c3c')
ax2.set_xlabel('nlist (clusters)')
ax2.set_ylabel('Search Time (ms)')
ax2.set_title('Search Time vs nlist')
ax2.set_xscale('log', base=2)
ax2.grid(True, alpha=0.3)

# Train time vs nlist
ax3 = axes[1, 0]
ax3.plot(df_nlist['nlist'], df_nlist['train_time'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax3.set_xlabel('nlist (clusters)')
ax3.set_ylabel('Training Time (s)')
ax3.set_title('Training Time vs nlist')
ax3.set_xscale('log', base=2)
ax3.grid(True, alpha=0.3)

# Vectors per list
ax4 = axes[1, 1]
ax4.plot(df_nlist['nlist'], df_nlist['vecs_per_list'], 'o-', linewidth=2, markersize=8, color='#9b59b6')
ax4.set_xlabel('nlist (clusters)')
ax4.set_ylabel('Vectors per List')
ax4.set_title('Average Cluster Size vs nlist')
ax4.set_xscale('log', base=2)
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

plt.suptitle(f'Effect of nlist (m={m_fixed}, nprobe={nprobe_fixed})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Experiment 3: Effect of nprobe (Search-time Parameter)

nprobe is the main search-time parameter:
- Higher nprobe → Better recall but slower search
- Can be adjusted at query time without rebuilding index
- Same trade-off as IVFFlat

In [ ]:
# Test different values of nprobe
nprobe_values = [1, 2, 4, 8, 16, 32, 64, 128, 256]
nlist_fixed = 256
m_fixed = 16

# Build index once
print(f"Building index with nlist={nlist_fixed}, m={m_fixed}...")
index, train_time, add_time = build_ivfpq_index(xb, nlist_fixed, m_fixed)
print(f"Index trained in {train_time:.2f}s, vectors added in {add_time:.2f}s\n")

results_nprobe = []

print("Experiment 3: Varying nprobe")
print(f"{'nprobe':>8} {'Search(ms)':>12} {'Recall':>10} {'QPS':>12} {'% Lists':>10}")
print("-" * 56)

for nprobe in nprobe_values:
    if nprobe > nlist_fixed:
        continue
        
    # Search
    distances, labels, search_time = search_ivfpq_index(index, xq, k, nprobe)
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    qps = nq / search_time
    pct_lists = (nprobe / nlist_fixed) * 100
    
    results_nprobe.append({
        'nprobe': nprobe,
        'search_time_ms': search_time * 1000,
        'recall': recall,
        'qps': qps,
        'pct_lists': pct_lists
    })
    
    print(f"{nprobe:>8} {search_time*1000:>12.2f} {recall:>10.4f} {qps:>12.0f} {pct_lists:>10.1f}%")

df_nprobe = pd.DataFrame(results_nprobe)

In [ ]:
# Visualize effect of nprobe
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Recall vs nprobe
ax1 = axes[0]
ax1.plot(df_nprobe['nprobe'], df_nprobe['recall'], 'o-', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('nprobe')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs nprobe')
ax1.set_xscale('log', base=2)
ax1.set_ylim([0, 1.02])
ax1.grid(True, alpha=0.3)

# Search time vs nprobe
ax2 = axes[1]
ax2.plot(df_nprobe['nprobe'], df_nprobe['search_time_ms'], 'o-', linewidth=2, markersize=8, color='#e74c3c')
ax2.set_xlabel('nprobe')
ax2.set_ylabel('Search Time (ms)')
ax2.set_title('Search Time vs nprobe')
ax2.set_xscale('log', base=2)
ax2.grid(True, alpha=0.3)

# QPS vs nprobe
ax3 = axes[2]
ax3.plot(df_nprobe['nprobe'], df_nprobe['qps'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax3.set_xlabel('nprobe')
ax3.set_ylabel('Queries Per Second')
ax3.set_title('Throughput vs nprobe')
ax3.set_xscale('log', base=2)
ax3.grid(True, alpha=0.3)

plt.suptitle(f'Effect of nprobe (nlist={nlist_fixed}, m={m_fixed})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Recall-QPS trade-off curve (Pareto frontier)
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(df_nprobe['recall'], df_nprobe['qps'], 'o-', linewidth=2, markersize=10, color='#8e44ad')

# Annotate points with nprobe values
for i, row in df_nprobe.iterrows():
    ax.annotate(f"nprobe={int(row['nprobe'])}", 
                (row['recall'], row['qps']),
                textcoords="offset points", 
                xytext=(5, 5), 
                fontsize=9)

ax.set_xlabel('Recall@10', fontsize=12)
ax.set_ylabel('Queries Per Second (QPS)', fontsize=12)
ax.set_title('Recall vs Throughput Trade-off\n(Pareto Frontier for nprobe)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Experiment 4: Combined Grid Search (m × nprobe)

Let's explore how m and nprobe interact at search time.

In [ ]:
# Grid search over m and nprobe
m_grid = [8, 16, 32, 64]
nprobe_grid = [4, 8, 16, 32, 64]
nlist_fixed = 256

# Pre-build indexes for each m
indexes = {}
print("Building indexes...")
for m in m_grid:
    index, train_time, add_time = build_ivfpq_index(xb, nlist_fixed, m)
    indexes[m] = index
    print(f"  m={m}: trained in {train_time:.2f}s, added in {add_time:.2f}s")

# Run grid search
grid_results = []

print("\nRunning grid search...")
for m in m_grid:
    index = indexes[m]
    for nprobe in nprobe_grid:
        distances, labels, search_time = search_ivfpq_index(index, xq, k, nprobe)
        recall = compute_recall(labels_gt, labels, k)
        qps = nq / search_time
        memory = estimate_memory_usage_ivfpq(nb, d, nlist_fixed, m)
        
        grid_results.append({
            'm': m,
            'nprobe': nprobe,
            'recall': recall,
            'search_time_ms': search_time * 1000,
            'qps': qps,
            'memory_mb': memory
        })

df_grid = pd.DataFrame(grid_results)
print("Grid search complete!")

In [ ]:
# Visualize grid results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = plt.cm.viridis(np.linspace(0, 1, len(m_grid)))

# Recall curves for different m
ax1 = axes[0]
for i, m in enumerate(m_grid):
    data = df_grid[df_grid['m'] == m]
    ax1.plot(data['nprobe'], data['recall'], 'o-', 
             linewidth=2, markersize=8, color=colors[i], label=f'm={m}')
ax1.set_xlabel('nprobe')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs nprobe for Different m')
ax1.set_xscale('log', base=2)
ax1.set_ylim([0, 1.02])
ax1.legend()
ax1.grid(True, alpha=0.3)

# Recall vs QPS for different m
ax2 = axes[1]
for i, m in enumerate(m_grid):
    data = df_grid[df_grid['m'] == m]
    ax2.plot(data['recall'], data['qps'], 'o-', 
             linewidth=2, markersize=8, color=colors[i], label=f'm={m}')
ax2.set_xlabel('Recall@10')
ax2.set_ylabel('Queries Per Second')
ax2.set_title('Recall-Throughput Trade-off')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle(f'Combined Effect of m and nprobe (nlist={nlist_fixed})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Heatmaps for grid search results
pivot_recall = df_grid.pivot(index='m', columns='nprobe', values='recall')
pivot_qps = df_grid.pivot(index='m', columns='nprobe', values='qps')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Recall heatmap
im1 = axes[0].imshow(pivot_recall.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1.0)
axes[0].set_xticks(range(len(nprobe_grid)))
axes[0].set_xticklabels(nprobe_grid)
axes[0].set_yticks(range(len(m_grid)))
axes[0].set_yticklabels(m_grid)
axes[0].set_xlabel('nprobe')
axes[0].set_ylabel('m')
axes[0].set_title('Recall@10 Heatmap')

# Add text annotations
for i in range(len(m_grid)):
    for j in range(len(nprobe_grid)):
        val = pivot_recall.values[i, j]
        axes[0].text(j, i, f'{val:.3f}', ha='center', va='center', color='black', fontsize=10)

fig.colorbar(im1, ax=axes[0])

# QPS heatmap
im2 = axes[1].imshow(pivot_qps.values, cmap='YlOrRd_r', aspect='auto')
axes[1].set_xticks(range(len(nprobe_grid)))
axes[1].set_xticklabels(nprobe_grid)
axes[1].set_yticks(range(len(m_grid)))
axes[1].set_yticklabels(m_grid)
axes[1].set_xlabel('nprobe')
axes[1].set_ylabel('m')
axes[1].set_title('QPS Heatmap')

# Add text annotations
for i in range(len(m_grid)):
    for j in range(len(nprobe_grid)):
        val = pivot_qps.values[i, j]
        axes[1].text(j, i, f'{val:.0f}', ha='center', va='center', color='black', fontsize=10)

fig.colorbar(im2, ax=axes[1])

plt.suptitle('Parameter Grid Search Results (m × nprobe)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Experiment 5: IVFPQ vs IVFFlat Comparison

Let's compare IVFPQ with IVFFlat to understand the memory-accuracy trade-off.

In [ ]:
# Compare IVFPQ with IVFFlat and Brute Force
print("Comparing IVFPQ with IVFFlat and Brute Force\n")

# Brute force baseline
index_flat = faiss.IndexFlatL2(d)
index_flat.add(xb)
start = time.time()
D_flat, I_flat = index_flat.search(xq, k)
flat_search_time = time.time() - start
flat_qps = nq / flat_search_time
flat_memory = d * nb * 4 / (1024 * 1024)

# IVFFlat
nlist_test = 256
quantizer_ivf = faiss.IndexFlatL2(d)
index_ivfflat = faiss.IndexIVFFlat(quantizer_ivf, d, nlist_test)
index_ivfflat.train(xb)
index_ivfflat.add(xb)
index_ivfflat.nprobe = 16
start = time.time()
D_ivfflat, I_ivfflat = index_ivfflat.search(xq, k)
ivfflat_search_time = time.time() - start
ivfflat_recall = compute_recall(I_flat, I_ivfflat, k)
ivfflat_qps = nq / ivfflat_search_time
ivfflat_memory = estimate_memory_usage_flat(nb, d)

# IVFPQ with different m values
comparison_results = [
    {'Method': 'Brute Force', 'Recall': 1.0, 'QPS': flat_qps, 'Memory (MB)': flat_memory},
    {'Method': 'IVFFlat', 'Recall': ivfflat_recall, 'QPS': ivfflat_qps, 'Memory (MB)': ivfflat_memory},
]

for m in [8, 16, 32, 64]:
    index_ivfpq, _, _ = build_ivfpq_index(xb, nlist_test, m)
    index_ivfpq.nprobe = 16
    start = time.time()
    D_ivfpq, I_ivfpq = index_ivfpq.search(xq, k)
    ivfpq_search_time = time.time() - start
    ivfpq_recall = compute_recall(I_flat, I_ivfpq, k)
    ivfpq_qps = nq / ivfpq_search_time
    ivfpq_memory = estimate_memory_usage_ivfpq(nb, d, nlist_test, m)
    
    comparison_results.append({
        'Method': f'IVFPQ (m={m})',
        'Recall': ivfpq_recall,
        'QPS': ivfpq_qps,
        'Memory (MB)': ivfpq_memory
    })

df_comparison = pd.DataFrame(comparison_results)
print(df_comparison.to_string(index=False))

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

methods = df_comparison['Method'].tolist()
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c']

# Memory comparison (bar chart)
ax1 = axes[0]
bars = ax1.bar(range(len(methods)), df_comparison['Memory (MB)'], color=colors[:len(methods)], edgecolor='black')
ax1.set_xticks(range(len(methods)))
ax1.set_xticklabels(methods, rotation=45, ha='right')
ax1.set_ylabel('Memory (MB)')
ax1.set_title('Memory Usage Comparison')
for i, v in enumerate(df_comparison['Memory (MB)']):
    ax1.text(i, v + 1, f'{v:.1f}', ha='center', fontsize=9)

# Recall comparison
ax2 = axes[1]
bars = ax2.bar(range(len(methods)), df_comparison['Recall'], color=colors[:len(methods)], edgecolor='black')
ax2.set_xticks(range(len(methods)))
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.set_ylabel('Recall@10')
ax2.set_title('Recall Comparison')
ax2.set_ylim([0, 1.1])
for i, v in enumerate(df_comparison['Recall']):
    ax2.text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=9)

# QPS comparison
ax3 = axes[2]
bars = ax3.bar(range(len(methods)), df_comparison['QPS'], color=colors[:len(methods)], edgecolor='black')
ax3.set_xticks(range(len(methods)))
ax3.set_xticklabels(methods, rotation=45, ha='right')
ax3.set_ylabel('Queries Per Second')
ax3.set_title('Search Speed Comparison')
for i, v in enumerate(df_comparison['QPS']):
    ax3.text(i, v + 100, f'{v:.0f}', ha='center', fontsize=9)

plt.suptitle('IVFPQ vs IVFFlat vs Brute Force (nlist=256, nprobe=16)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Memory-Recall trade-off scatter plot
fig, ax = plt.subplots(figsize=(10, 6))

scatter = ax.scatter(df_comparison['Memory (MB)'], df_comparison['Recall'],
                     s=df_comparison['QPS'] / 50,  # Size based on QPS
                     c=range(len(df_comparison)),
                     cmap='tab10', alpha=0.7, edgecolors='black', linewidth=2)

# Add labels
for i, row in df_comparison.iterrows():
    ax.annotate(row['Method'], 
                (row['Memory (MB)'], row['Recall']),
                textcoords="offset points",
                xytext=(10, 5),
                fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

ax.set_xlabel('Memory (MB)', fontsize=12)
ax.set_ylabel('Recall@10', fontsize=12)
ax.set_title('Memory vs Recall Trade-off\n(Bubble size = QPS)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print compression ratios
print("\nCompression ratios vs IVFFlat:")
for _, row in df_comparison.iterrows():
    if 'IVFPQ' in row['Method']:
        ratio = ivfflat_memory / row['Memory (MB)']
        print(f"  {row['Method']}: {ratio:.1f}x smaller, recall={row['Recall']:.3f}")

## 9. Experiment 6: Typical Configuration Examples

Let's test some commonly recommended configurations for different use cases.

In [ ]:
# Typical configurations for different use cases
configs = [
    {'name': 'High compression', 'nlist': 256, 'm': 8, 'nprobe': 8},
    {'name': 'Balanced (memory)', 'nlist': 256, 'm': 16, 'nprobe': 16},
    {'name': 'Balanced (recall)', 'nlist': 128, 'm': 32, 'nprobe': 16},
    {'name': 'High recall', 'nlist': 100, 'm': 64, 'nprobe': 32},
    {'name': 'Speed optimized', 'nlist': 1024, 'm': 16, 'nprobe': 8},
]

config_results = []

print("Experiment 6: Typical Configuration Examples\n")
print(f"{'Config':>20} {'nlist':>6} {'m':>4} {'nprobe':>7} {'Train(s)':>10} {'Search(ms)':>12} {'Recall':>8} {'Mem(MB)':>10}")
print("-" * 90)

for config in configs:
    # Build index
    index, train_time, add_time = build_ivfpq_index(xb, config['nlist'], config['m'])
    
    # Search
    distances, labels, search_time = search_ivfpq_index(index, xq, k, config['nprobe'])
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    qps = nq / search_time
    memory = estimate_memory_usage_ivfpq(nb, d, config['nlist'], config['m'])
    
    config_results.append({
        'name': config['name'],
        'nlist': config['nlist'],
        'm': config['m'],
        'nprobe': config['nprobe'],
        'train_time': train_time,
        'search_time_ms': search_time * 1000,
        'recall': recall,
        'qps': qps,
        'memory_mb': memory
    })
    
    print(f"{config['name']:>20} {config['nlist']:>6} {config['m']:>4} {config['nprobe']:>7} "
          f"{train_time:>10.2f} {search_time*1000:>12.2f} {recall:>8.4f} {memory:>10.1f}")

df_configs = pd.DataFrame(config_results)

In [ ]:
# Visualize configurations
fig, ax = plt.subplots(figsize=(12, 7))

# Scatter plot: recall vs memory, color by QPS
scatter = ax.scatter(df_configs['memory_mb'], df_configs['recall'],
                     s=df_configs['qps'] / 30,  # Size based on QPS
                     c=df_configs['m'],  # Color based on m
                     cmap='coolwarm', alpha=0.7, edgecolors='black', linewidth=2)

# Add labels
for i, row in df_configs.iterrows():
    ax.annotate(f"{row['name']}\nnlist={row['nlist']}, m={row['m']}, nprobe={row['nprobe']}",
                (row['memory_mb'], row['recall']),
                textcoords="offset points",
                xytext=(10, 5),
                fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

ax.set_xlabel('Memory (MB)', fontsize=12)
ax.set_ylabel('Recall@10', fontsize=12)
ax.set_title('Typical IVFPQ Configurations\n(Size = QPS, Color = m)', fontsize=14, fontweight='bold')

cbar = plt.colorbar(scatter)
cbar.set_label('m (subquantizers)', fontsize=11)

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Summary and Recommendations

Based on our experiments, here are the key findings:

In [ ]:
print("="*70)
print("IVFPQ Parameter Tuning Guidelines")
print("="*70)

print(f"""
INDEX CREATION PARAMETERS:

1. m (Number of subquantizers):
   - Higher m → Better accuracy, more memory
   - Must divide dimension d (here d={d})
   - Typical values: 8, 16, 32, 64
   - With nbits=8: each vector uses m bytes
   - Compression ratio = (d * 4 bytes) / m bytes = {d*4}/m
   
   Recommended m values for different needs:
   - High compression (16-32x): m = 4-8
   - Balanced: m = 16-32
   - High accuracy: m = 64 or more

2. nlist (Number of clusters):
   - Same guidelines as IVFFlat
   - Rule of thumb: sqrt(n) where n = database size
   - For {nb:,} vectors: sqrt({nb}) ≈ {int(np.sqrt(nb))}

3. nbits (Bits per subquantizer):
   - Usually 8 (256 centroids per subspace)
   - Can be 4 for extreme compression (but lower accuracy)

SEARCH-TIME PARAMETERS:

4. nprobe (Clusters to search):
   - Main recall/speed trade-off
   - Can be adjusted without rebuilding
   - Typical: 1-10% of nlist

Common Configurations:
---------------------------------------------------------------
| Use Case              | nlist    | m  | nprobe | Compression|
|-----------------------|----------|----|---------|-----------:|
| High compression      | sqrt(n)  | 8  | 4-8    | 64x        |
| Balanced              | sqrt(n)  | 16 | 8-16   | 32x        |
| High recall           | sqrt(n)/2| 32 | 16-32  | 16x        |
| Very high recall      | sqrt(n)/4| 64 | 32-64  | 8x         |
---------------------------------------------------------------
""")

# Show best configurations from experiments
print("\nBest configurations from our experiments:")
print("-" * 50)

# Find highest recall configuration
best_recall = df_grid.loc[df_grid['recall'].idxmax()]
print(f"Highest recall: m={int(best_recall['m'])}, nprobe={int(best_recall['nprobe'])}")
print(f"  Recall: {best_recall['recall']:.4f}, QPS: {best_recall['qps']:.0f}")

# Find best QPS with >80% recall
good_recall = df_grid[df_grid['recall'] > 0.80].sort_values('qps', ascending=False)
if len(good_recall) > 0:
    best = good_recall.iloc[0]
    print(f"\nFastest with >80% recall: m={int(best['m'])}, nprobe={int(best['nprobe'])}")
    print(f"  Recall: {best['recall']:.4f}, QPS: {best['qps']:.0f}")

In [ ]:
def test_ivfpq_config(nlist, m, nprobe, xb, xq, labels_gt, k, nbits=8):
    """
    Test a specific IVFPQ configuration and print results.
    """
    d = xb.shape[1]
    if d % m != 0:
        print(f"Error: m={m} does not divide d={d}. Valid m values: {[i for i in [1,2,4,8,16,32,64] if d % i == 0]}")
        return None, None, None
    
    print(f"Testing IVFPQ with nlist={nlist}, m={m}, nprobe={nprobe}, nbits={nbits}")
    print("-" * 60)
    
    # Build
    index, train_time, add_time = build_ivfpq_index(xb, nlist, m, nbits)
    print(f"Training time: {train_time:.2f}s")
    print(f"Add time: {add_time:.2f}s")
    
    # Search
    distances, labels, search_time = search_ivfpq_index(index, xq, k, nprobe)
    print(f"Search time: {search_time*1000:.2f}ms for {len(xq)} queries")
    
    # Metrics
    recall = compute_recall(labels_gt, labels, k)
    qps = len(xq) / search_time
    memory = estimate_memory_usage_ivfpq(len(xb), d, nlist, m, nbits)
    compression = (d * 4) / m
    
    print(f"\nRecall@{k}: {recall:.4f}")
    print(f"QPS: {qps:.0f}")
    print(f"Memory: {memory:.1f} MB")
    print(f"Compression ratio: {compression:.1f}x")
    print(f"Bytes per vector: {m} (from {d*4} bytes)")
    
    return index, recall, qps

# Example: Try your own configuration!
# Modify these parameters and run the cell
my_nlist = 200
my_m = 32  # Must divide d=128
my_nprobe = 20

test_ivfpq_config(my_nlist, my_m, my_nprobe, xb, xq, labels_gt, k)

## 12. Bonus: Using index_factory String

FAISS provides a convenient `index_factory` function that creates indexes from a string description.

In [ ]:
# Using index_factory for IVFPQ
print("Creating IVFPQ indexes using index_factory:\n")

# Different index factory strings
# Format: IVF{nlist},PQ{m} or IVF{nlist},PQ{m}x{nbits}
factory_strings = [
    "IVF256,PQ8",       # 256 clusters, PQ with 8 subquantizers
    "IVF256,PQ16",      # 256 clusters, PQ with 16 subquantizers
    "IVF256,PQ32",      # 256 clusters, PQ with 32 subquantizers
    "IVF100,PQ16",      # 100 clusters (for higher recall)
]

for factory_string in factory_strings:
    print(f"Factory string: '{factory_string}'")
    
    # Create index using factory
    index = faiss.index_factory(d, factory_string)
    
    # Train and add
    start = time.time()
    index.train(xb)
    train_time = time.time() - start
    
    start = time.time()
    index.add(xb)
    add_time = time.time() - start
    
    # Search with nprobe=16
    index.nprobe = 16
    start = time.time()
    D, I = index.search(xq, k)
    search_time = time.time() - start
    
    recall = compute_recall(labels_gt, I, k)
    
    print(f"  nlist: {index.nlist}, nprobe: {index.nprobe}")
    print(f"  Train: {train_time:.2f}s, Add: {add_time:.2f}s, Search: {search_time*1000:.2f}ms")
    print(f"  Recall@10: {recall:.4f}")
    print()

## 13. Bonus: OPQ (Optimized Product Quantization)

OPQ applies a rotation to vectors before PQ to improve accuracy.

In [ ]:
# Compare IVFPQ with OPQ (Optimized Product Quantization)
print("Comparing IVFPQ with OPQ:\n")

# Standard IVFPQ
index_ivfpq = faiss.index_factory(d, "IVF256,PQ16")
index_ivfpq.train(xb)
index_ivfpq.add(xb)
index_ivfpq.nprobe = 16
D_ivfpq, I_ivfpq = index_ivfpq.search(xq, k)
recall_ivfpq = compute_recall(labels_gt, I_ivfpq, k)

# IVFPQ with OPQ preprocessing
# OPQ{m} applies a rotation learned to minimize PQ quantization error
index_opq = faiss.index_factory(d, "OPQ16,IVF256,PQ16")
index_opq.train(xb)
index_opq.add(xb)
# For OPQ index, nprobe is set on the IVF part
faiss.downcast_index(index_opq.index).nprobe = 16
D_opq, I_opq = index_opq.search(xq, k)
recall_opq = compute_recall(labels_gt, I_opq, k)

print(f"IVFPQ (IVF256,PQ16):")
print(f"  Recall@10: {recall_ivfpq:.4f}")

print(f"\nOPQ+IVFPQ (OPQ16,IVF256,PQ16):")
print(f"  Recall@10: {recall_opq:.4f}")

print(f"\nOPQ improvement: +{(recall_opq - recall_ivfpq)*100:.1f}% recall")